In [1]:
import numpy as np

from utils.data import create_data
from utils.utils import scikit_svm, predict, accuracy, rbf_kernel
from svm import SVM

In [2]:
#create_data(file_name = 'data_38', classes = [3,8])
load_data = np.load('data_38.npz')
train_X, train_y = load_data['train_X'], load_data['train_y']
test_X, test_y = load_data['test_X'], load_data['test_y']

# Checking if data is linearly separable

In [10]:
# Takes a bit to run
#linear = scikit_svm('linear', train_X, train_y, test_X, test_y)
#rbf = scikit_svm('rbf', train_X, train_y, test_X, test_y)
print(f'Accuracy using linear kernel: {linear}')
print(f'Accuracy using gaussian kernel: {rbf}')

Accuracy using linear kernel: 0.9702620967741935
Accuracy using gaussian kernel: 0.9924395161290323


In [3]:
# Create mini dataset for testing
train_X, train_y = train_X[:1000,:], train_y[:1000]
test_X, test_y = test_X[:500,:], test_y[:500]

# Running SMO

## Get accuracy before training

In [8]:
num_training_examples = train_y.shape[0]

alpha = np.ones(num_training_examples)
b = 0
kernel_matrix = rbf_kernel(train_X, test_X)

pred_y = predict(alpha, kernel_matrix, b, train_y)

print(f'Accuracy before training: {accuracy(pred_y, test_y)}')

Accuracy before training: 0.5090725806451613


In [4]:
svm = SVM(optim = 'SMO',
          kernel = 'rbf',
          C = 1)

In [5]:
svm.fit(train_X, train_y)

In [9]:
print(svm.get_accuracy(test_X, test_y))

0.94


In [8]:
len(svm.alpha)

338